#0. Setting up the Environment
Runtime -> Change runtime type -> Hardware accelerator -> GPU

In [ ]:
! pip install pydicom torchmetrics pytorch_lightning
! pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pydicom-2.3.1-py3-none-any.whl (2.0 MB)
  Using cached torchmetrics-0.11.4-py3-none-any.whl (519 kB)
  Using cached pytorch_lightning-2.0.2-py3-none-any.whl (719 kB)
  Using cached lightning_utilities-0.8.0-py3-none-any.whl (20 kB)
  Using cached aiohttp-3.8.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
  Using cached multidict-6.0.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (114 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Imports for this project
import json 
import os
from pathlib import Path
import zipfile
import os
from google.colab import drive

import numpy as np
import pandas as pd
import seaborn as sns
import imageio
import matplotlib.pyplot as plt
import matplotlib

import pydicom
from pydicom import dcmread

import cv2
import torchvision
import torchmetrics
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, Dataset
from PIL import Image

import torch
from torch.utils import data
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm

import random

#1. Downloading Data


In this section we download the data needed for our poject from Kaggle. The dataset can be found [here](https://www.kaggle.com/c/rsna-pneumonia-detection-challenge).

Work done by: Aidan Daly


In [ ]:
# API Username and Key
api_key = {
'username':"aidandaly" ,
'key':"9f430a42e2d60b123bead1ea61ad5617"}

#Sets up data download
kaggle_path = Path('/root/.kaggle')
os.makedirs(kaggle_path, exist_ok=True)
with open (kaggle_path/'kaggle.json', 'w') as handl:
  json.dump(api_key,handl)

os.chmod(kaggle_path/'kaggle.json', 600) 

In [ ]:
! kaggle competitions download -c rsna-pneumonia-detection-challenge

In [ ]:
# Unzip downloaded file from kaggle into a new folder "project"
with zipfile.ZipFile("rsna-pneumonia-detection-challenge.zip", 'r') as zip_ref:
  zip_ref.extractall("./project")

In [ ]:
# Check to make sure everything was downloaded correctly
print(os.listdir("./project"))

#2a. Visualizing Data

In this section we will attempt to look through the data and gain a better understanding of it through some visualizations.

Work done by: Aidan Daly

In [ ]:
label_data = pd.read_csv("./project/stage_2_train_labels.csv")
label_data.drop_duplicates()
columns = ['patientId', 'Target']

label_data = label_data.filter(columns)
label_data.head(5)

We can see the data is unbalanced, but we'll deal with that later.

In [ ]:
sns.set(style='whitegrid')

# Here I change the label 0 to "Pneumonia Free Tissue" and 1 to "Pneumonia Tissue"
pie_chart=pd.DataFrame(label_data['Target'].replace(0,'Pneumonia Free Tissue').replace(1,'Pneumonia Tissue').value_counts())
pie_chart.reset_index(inplace=True)

# Print out the chart
pie_chart.plot(kind='pie', title='Image Labels',y = 'Target', 
             autopct='%1.1f%%', shadow=False, labels=pie_chart['index'], legend = True, fontsize=15, figsize=(18,8))


We can now visualize some of the data here.

In [ ]:
fig, ax = plt.subplots(3, 3, figsize=(10, 10))
for i, axis in enumerate(ax.flat):
  im_file = str("./project/stage_2_train_images/" + label_data.patientId[i] + '.dcm')
  im_dcm = dcmread(im_file)
  axis.imshow(im_dcm.pixel_array, cmap="bone")
  axis.set(xticks=[], yticks=[], xlabel = label_data.Target[i]);

#2b. Processing Data
Below we begin to process our data. We split our training data into training and validation.

Directly below you can see a class called `Dataset` this is something we implemented from a similar Kaggle project.

Work Done By: Aidan Daly

In [ ]:
# Dataset
class Dataset(data.Dataset):
    
    def __init__(self, paths, labels=None, transform=None):
        self.paths = paths
        self.labels = labels
        self.transform = transform
    
    def __getitem__(self, index):
        image = dcmread(f'{self.paths[index]}.dcm')
        image = image.pixel_array
        image = image / 255.0

        image = (255*image).clip(0, 255).astype(np.uint8)
        image = Image.fromarray(image).convert('RGB')

        label = self.labels[index][1]
        
        if self.transform is not None:
            image = self.transform(image)
            
        return image, label
    
    def __len__(self):
        
        return len(self.paths)

In [ ]:
def balance(imgs):
  remove_dups(imgs)
  new_imgs = []
  zero_count = 0
  one_count = 0
  for i in range(len(imgs)):
    if imgs[i][1] == 0:
      zero_count += 1
    else:
      one_count += 1
  balancee = 0
  if one_count > zero_count:
    balancee = 1

  diff = abs(zero_count - one_count)
  removed = 0
  for i in range(len(imgs)):
    if imgs[i][1] != balancee or removed >= diff:
      new_imgs.append(imgs[i])
    else:
      removed += 1

  return np.asarray(new_imgs)


def remove_dups(imgs):
  new_imgs = []
  seen = set()
  for i in range(len(imgs)):
    if imgs[i][0] not in seen:
      seen.add(imgs[i][0])
      new_imgs.append(imgs[i])
  return np.asarray(new_imgs)
  

In [ ]:
train_f = './project/stage_2_train_images'
test_f = './project/stage_2_test_images'

#Split Data
train_labels, val_labels = train_test_split(label_data.values, test_size=0.1)
train_labels, test_labels = train_test_split(train_labels, test_size=0.01)
print(f"Trining Data Shape: {train_labels.shape}")
print(f"Validation Data Shape: {val_labels.shape}")

train_labels = balance(train_labels)
val_labels = balance(val_labels)
test_labels= balance(test_labels)
print(f"Balanced Trining Data Shape: {train_labels.shape}")
print(f"Balanced Validation Data Shape: {val_labels.shape}")
print(f"Balanced Test Data Shape: {test_labels.shape}")

train_paths = [os.path.join(train_f, image[0]) for image in train_labels]
val_paths = [os.path.join(train_f, image[0]) for image in val_labels]
test_paths = [os.path.join(train_f, image[0]) for image in test_labels]

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.Resize(224),
    transforms.ToTensor()])


#Check Dataset
train_dataset = Dataset(train_paths, train_labels, transform=transform)
image = iter(train_dataset)
img, label = next(image)
img = np.transpose(img, (1, 2, 0))
plt.imshow(img)

#Create Datasets and DataLoaders
train_dataset = Dataset(train_paths, train_labels, transform=transform)
val_dataset = Dataset(val_paths, val_labels, transform=transform)
test_dataset = Dataset(test_paths, test_labels, transform = transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)
grad_cam_loader = DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)

In [ ]:
#Kaggle Test Setup
kaggle_labels = os.listdir('./project/stage_2_test_images')
for i in range(len(kaggle_labels)):
  kaggle_labels[i] = [kaggle_labels[i][:-4], 0]
kaggle_paths = [os.path.join(test_f,image[0]) for image in kaggle_labels]

kaggle_test_dataset = Dataset(kaggle_paths, kaggle_labels, transform = transform)
test_dataset_loader = DataLoader(dataset = kaggle_test_dataset,  batch_size=32)


#3. Model
For this model we decided to use a commonly used Convolutional Neural Network (CNN) called ResNet18. We decided to essentially use the base model as it is sufficient enough for what we are trying to do.

Our model is also inspired by a few different projects we found online, which we combined together with our own knowledge as well as previous works in Psets 3 and 4.

Work done by: Aidan Daly

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


class GradCamModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.gradients = None
        self.tensorhook = []
        self.layerhook = []
        self.selected_out = None
        
        # MODEL
        self.model = torchvision.models.resnet50(weights="IMAGENET1K_V2")
        self.layerhook.append(self.model.layer4.register_forward_hook(self.forward_hook()))

        
        for p in self.model.parameters():
            p.requires_grad = True

        
    
    def activations_hook(self,grad):
        self.gradients = grad

    def get_act_grads(self):
        return self.gradients

    def forward_hook(self):
        def hook(module, inp, out):
            self.selected_out = out
            self.tensorhook.append(out.register_hook(self.activations_hook))
        return hook

    def forward(self,x):
        out = self.model(x)
        return out, self.selected_out


#Model
model = GradCamModel()
model.to(device)

#Loss
criterion = nn.CrossEntropyLoss()


#Optimization
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

#4a. Training
You must run the `check_accuracy()` function, but we do not recommend running training as it may use up all cuda memory which will be needed as we continue.

Work done by: Luke Pisani and Aidan Daly

In [ ]:
def check_accuracy(model, loader):
  print('Checking accuracy on current set')   
  # Validation step
  correct = 0
  total = 0  
  for images, labels in tqdm(loader):
      images = images.to(device)
      labels = labels.to(device)
      predictions, x = model(images)
      _, predicted = torch.max(predictions, 1)
      total += labels.size(0)
      correct += (labels == predicted).sum()
  val_acc= 100*correct/total
  return val_acc

In [ ]:
# drive.mount('/content/drive')

num_epochs = 20
# Train the model
total_step = len(train_loader)

for epoch in range(num_epochs):
    # Training step
    for i, (images, labels) in tqdm(enumerate(train_loader)):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs, x = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 2000 == 0:
            
            print("Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


    # Validation step
    val_acc=check_accuracy(model, val_loader)
    print(f'Epoch: {epoch+1}/{num_epochs}, Val_Acc: {val_acc}')
    if epoch %5 == 0:
      model_save_name = 'classifier.pth'
      path = f"drive/MyDrive/{model_save_name}" 
      torch.save(model.state_dict(), path)

#4b. Grad Cam Visualization
We will now inspect what the grad cam heatmap looks like for specific images. It's clear to see that the heat map is looking in the area of the lungs.

In [ ]:
! gdown 1ctysbvJRG86cbDkha70ldUj6zCWS3UwL
! ls

In [ ]:
import matplotlib as mpl
from skimage.transform import resize

# set the evaluation mode
path = "/content/classifier.pth"
model.load_state_dict(torch.load(path, map_location=torch.device(device)))
model.eval()
i = 0
# get the image from the dataloader
for images, labels in tqdm(grad_cam_loader):
    images = images.to(device)
    labels = labels.to(device)

    img = images.cpu().detach().numpy()
    img /= 255.0
    mean = np.array([0.485, 0.456, 0.406]).reshape((1,3,1,1))
    std = np.array([0.229, 0.224, 0.225]).reshape((1,3,1,1))

    out, acts = model(images)
    i += 1
    if i == 1:
      break
acts = acts.detach().cpu()

loss = nn.CrossEntropyLoss()(out,torch.from_numpy(np.array([600])).to(device))
loss.backward()

grads = model.get_act_grads().detach().cpu()

pooled_grads = torch.mean(grads, dim=[0,2,3]).detach().cpu()

for i in range(acts.shape[1]):
    acts[:,i,:,:] *= pooled_grads[i]

heatmap_j = torch.mean(acts, dim = 1).squeeze()
heatmap_j_max = heatmap_j.max(axis = 0)[0]
heatmap_j /= heatmap_j_max

cmap = mpl.colormaps.get_cmap('plasma')
heatmap_j2 = cmap(heatmap_j,alpha = 0.2)

fig, axs = plt.subplots(1,1,figsize = (5,5))
axs.imshow(((img*std+mean)[0].transpose(1,2,0)))
axs.imshow(heatmap_j2)
plt.show()

#5. Testing
Checking our acccuracy on our Test Dataset

Author: Luke Pisani

In [ ]:

path = "/content/classifier.pth"
model.load_state_dict(torch.load(path, map_location=torch.device(device)))
acc =check_accuracy(model, test_loader)
print(f"\nAccuracy on Test: {acc}")


Here we create our sample submission for Kaggle

In [ ]:
unlabeled = pd.read_csv('./project/stage_2_sample_submission.csv')
model.eval()

predictions = []

for i, (images, labels) in enumerate(tqdm(test_dataset_loader, total=int(len(test_dataset_loader)))):
  images = images.to(device)
  labels = labels.to(device)
  
  pred, outputs = model(images)
  _, predicted = torch.max(pred, 1)
  
  for j in predicted:
      predictions.append(j.item())


unlabeled['PredictionString'] = predictions


test_images = np.random.choice(unlabeled.patientId, size=50, replace=False)     

fig, ax = plt.subplots(5, 10, figsize=(20,10))

for n in range(5):
  for m in range(10):
    img_id = test_images[m + n*10]
    image = dcmread(test_f + '/' + img_id + ".dcm").pixel_array
    pred = unlabeled.loc[unlabeled['patientId'] == img_id, 'PredictionString'].values[0]
    label = "Pneumonia" if(pred >= 0.5) else "Healthy"  
    ax[n,m].imshow(image, cmap="bone")
    ax[n,m].grid(False)
    ax[n,m].tick_params(labelbottom=False, labelleft=False)
    ax[n,m].set_title("Label: " + label)
      

#6. Filter Visualization
Below we get a random image from the dataset and push it through the model, saving the filter map after each layer of convolution. This helps to get a sense of how each layer is modifying an input image.

Code adapted from https://ravivaishnav20.medium.com/visualizing-feature-maps-using-pytorch-12a48cd1e573

Work done by: Aidan Daly

In [ ]:
# we will save the conv layers and weights in these lists
model_weights =[]
conv_layers = []

# get all the model children as list
model_children = list(model.model.children())

counter = 0
# append all the conv layers and their respective weights to the list
for i in range(len(model_children)):
    if type(model_children[i]) == nn.Conv2d:
        counter+=1
        model_weights.append(model_children[i].weight)
        conv_layers.append(model_children[i])
    elif type(model_children[i]) == nn.Sequential:
        for j in range(len(model_children[i])):
            for child in model_children[i][j].children():
                if type(child) == nn.Conv2d:
                    counter+=1
                    model_weights.append(child.weight)
                    conv_layers.append(child)
print(f"Total convolution layers: {counter}")
print("conv_layers")

In [ ]:
# transform an individual image to something we can push through the model
transform = transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize(mean=0., std=1.)
])

In [ ]:
# get a random image
idx = random.randrange(0, len(label_data.patientId), 1)
f = str("./project/stage_2_train_images/" + label_data.patientId[idx] + ".dcm")
image = dcmread(f)
plt.imshow(image.pixel_array, cmap="bone")
plt.axis('off')
if label_data.Target[idx] == 1:
  plt.title("Pneumonia")
else:
  plt.title("Healthy")

plt.show()

# transform and load to GPU if available
image = image.pixel_array
image = image / 255.0

image = (255*image).clip(0, 255).astype(np.uint8)
image = Image.fromarray(image).convert('RGB')
image = transform(image).to(device)
outputs = []
names = []
for layer in conv_layers[0:]:
  image = layer(image)
  outputs.append(image)
  names.append(str(layer))

print("number of conv2d layers:", len(outputs))

In [ ]:
processed = []
for feature_map in outputs:
  feature_map = feature_map.squeeze(0)
  gray_scale = torch.sum(feature_map,0)
  gray_scale = gray_scale / feature_map.shape[0]
  processed.append(gray_scale.data.cpu().numpy())

# display feature map after each conv2d layer
fig = plt.figure(figsize=(10, 10))
for i in range(12):
  a = fig.add_subplot(4, 3, i+1)
  imgplot = plt.imshow(processed[i], cmap="bone")
  a.axis("off")
  a.set_title(names[i].split('(')[0], fontsize=15)